In [1]:
# Nhập thư viện
import pandas as pd
import numpy as np
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import matplotlib.cm as cm
from wordcloud import WordCloud

import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

corpus = pd.read_csv('data/mental_health.csv')

# Tổng quát về quá trình dọn dữ liệu

### Quy trình:
1. Đảm bảo các văn bản chỉ chứa các chữ cái viết thường trong ngôn ngữ Tiếng Anh.
2. Loại bỏ các đường link (token có chứa http, www).
3. Loại bỏ các tokens dài hơn 16 ký tự.
4. Xóa ký tự 'i' ở cuối tại mọi tokens kết thúc bằng ký tự 'i'
5. Loại bỏ các tokens cụ thể trên cơ sở của phần phân tích n-Grams:
- filler
- br
- nowdrink
- pee
- poo
- eve
- click
- horny
- ampxb
- monitor
- Mọi tokens bắt đầu bằng 'gt'

6. Thực hiện Lemmatization.
7. Thực hiện Stop-words removal.
8. Đưa những yếu tố có char lặp lại về 1 char (eg. soooooo -> so, dddddd -> d).
9. Loại bỏ các token chỉ có 1 char.
10. Loại bỏ các token có tần suất nhỏ hơn 5.
11. Loại bỏ các sample được coi là dị thường dựa vào total tokens.
12. Loại bỏ các sample trống, chỉ còn whitespace.

#### Lưu ý: Quy trình này được lặp lại cho tới khi không còn thay đổi nào được thực hiện (4 lần)

In [38]:
# Bước 1: Đảm bảo các văn bản chỉ chứa các chữ cái viết thường trong ngôn ngữ Tiếng Anh
def remove_non_lowercase(text):
    text = re.sub(r'\s+', ' ', text)
    removed_chars = set(re.findall(r'[^a-z\s]', text))
    cleaned_text = re.sub(r'[^a-z\s]', '', text)
    return cleaned_text.strip(), removed_chars

corpus['text'], removed_chars = zip(*corpus['text'].apply(remove_non_lowercase))

removed_chars = set(char for chars in removed_chars for char in chars)

print("Unique characters removed:", removed_chars)

Unique characters removed: set()


In [39]:
# Bước 2: Loại bỏ các đường link (token có chứa http, www)
def remove_links(text):
    tokens = text.split()
    cleaned_tokens = [token for token in tokens if 'http' not in token and 'www' not in token]
    cleaned_text = ' '.join(cleaned_tokens)
    return cleaned_text

original_text = corpus['text'].copy()

corpus['text'] = corpus['text'].apply(remove_links)

affected_rows = (original_text != corpus['text']).sum()

print(f"Total number of rows affected: {affected_rows}")

Total number of rows affected: 0


In [40]:
# Bước 3: Loại bỏ các tokens dài hơn 16 ký tự
def remove_long_tokens(text):
    tokens = text.split()
    cleaned_tokens = [token for token in tokens if len(token) <= 16]
    return ' '.join(cleaned_tokens)

original_text = corpus['text'].copy()

corpus['text'] = corpus['text'].apply(remove_long_tokens)

removed_tokens_count = sum(len(original.split()) - len(cleaned.split()) for original, cleaned in zip(original_text, corpus['text']))

print(f"Total number of tokens removed: {removed_tokens_count}")

Total number of tokens removed: 0


In [41]:
# Bước 4: Xóa ký tự 'i' với mọi token kết thúc bằng ký tự 'i'
def strip_last_i_count_changes(text):
    tokens = text.split()
    changes_made = 0
    stripped_tokens = []
    
    for token in tokens:
        if token.endswith('i'):
            stripped_tokens.append(token[:-1])
            changes_made += 1
        else:
            stripped_tokens.append(token)
    
    return ' '.join(stripped_tokens), changes_made

corpus['text'], changes_made = zip(*corpus['text'].apply(strip_last_i_count_changes))

total_changes_made = sum(changes_made)

print(f"Total number of tokens where the last 'i' was removed: {total_changes_made}")

Total number of tokens where the last 'i' was removed: 0


In [42]:
# Bước 5: Loại bỏ các tokens cụ thể dựa trên cơ sở phần phân tích n-Grams
specific_words = ['filler', 'br', 'eve', 'pee', 'poo', 'ampxb', 'nowdrink', 'monitor', 'click', 'horny']

def remove_specific_tokens(text):
    tokens = text.split()
    cleaned_tokens = [token for token in tokens if token not in specific_words and not token.startswith('gt')]
    return ' '.join(cleaned_tokens)

original_text = corpus['text'].copy()

corpus['text'] = corpus['text'].apply(remove_specific_tokens)

affected_rows = (original_text != corpus['text']).sum()

print(f"Total number of rows affected: {affected_rows}")

Total number of rows affected: 0


In [43]:
# Bước 6: Thực hiện Lemmatization
# nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    tokens = text.split()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

original_text = corpus['text'].copy()
corpus['text'] = corpus['text'].apply(lemmatize_text)

affected_rows = (original_text != corpus['text']).sum()
print(f"Total number of rows affected by lemmatization: {affected_rows}")

Total number of rows affected by lemmatization: 0


In [44]:
# Bước 7: Loại bỏ stop-words
# nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def remove_stop_words(text):
    tokens = text.split()
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(filtered_tokens)

original_text = corpus['text'].copy()
corpus['text'] = corpus['text'].apply(remove_stop_words)

affected_rows = (original_text != corpus['text']).sum()
print(f"Total number of rows affected by stop words removal: {affected_rows}")

Total number of rows affected by stop words removal: 0


In [45]:
# Bước 8: Đưa những yếu tố có char lặp lại về 1 char (eg. soooooo -> so, dddddd -> d)
def handle_repeated_chars(text):
    cleaned_text = re.sub(r'(.)\1{2,}', r'\1', text)
    changes_made = sum(1 for original, cleaned in zip(text, cleaned_text) if original != cleaned)
    return cleaned_text, changes_made

original_text = corpus['text'].copy()
corpus['text'], changes_made = zip(*corpus['text'].apply(handle_repeated_chars))

total_changes = sum(changes_made)

print(f"Total changes made in repeated character handling: {total_changes}")

Total changes made in repeated character handling: 0


In [46]:
# Bước 9: loại bỏ các tokens chỉ có 1 char
def remove_one_char_tokens(text):
    tokens = text.split()
    cleaned_tokens = [token for token in tokens if len(token) > 1]
    changes_made = len(tokens) - len(cleaned_tokens)
    return ' '.join(cleaned_tokens), changes_made

original_text = corpus['text'].copy()
corpus['text'], one_char_changes = zip(*corpus['text'].apply(remove_one_char_tokens))

total_one_char_changes = sum(one_char_changes)
print(f"Total changes made in removing one-character tokens: {total_one_char_changes}")

Total changes made in removing one-character tokens: 0


In [47]:
# Bước 10: Loại bỏ các tokens có tần suất nhỏ hơn 5
all_tokens = ' '.join(corpus['text']).split()
token_freq = pd.Series(all_tokens).value_counts()
single_freq_tokens = set(token_freq[token_freq <= 4].index)

def remove_single_freq_tokens(text):
    tokens = text.split()
    cleaned_tokens = [token for token in tokens if token not in single_freq_tokens]
    changes_made = len(tokens) - len(cleaned_tokens)
    return ' '.join(cleaned_tokens), changes_made

original_text = corpus['text'].copy()
corpus['text'], single_freq_changes = zip(*corpus['text'].apply(remove_single_freq_tokens))

total_single_freq_changes = sum(single_freq_changes)
print(f"Total changes made in removing tokens with frequency less than 5: {total_single_freq_changes}")

Total changes made in removing tokens with frequency less than 5: 0


In [48]:
# Bước 11: Loại bỏ các sample được coi là dị thường dựa vào total tokens.
max_tokens = 1249

initial_row_count = len(corpus)
corpus = corpus[corpus['text'].apply(lambda x: len(x.split()) <= max_tokens)]
removed_rows = initial_row_count - len(corpus)

print(f"Total number of rows removed (longer than {max_tokens} tokens): {removed_rows}")

Total number of rows removed (longer than 1249 tokens): 0


In [49]:
# Bước 12: Loại bỏ các sample trống hoặc chỉ chứa whitespace
corpus = corpus[corpus['text'].str.strip() != '']

# Thông tin của bộ corpus đã dọn
print("Shape of the cleaned data:", corpus.shape)
print("\nInfo of the cleaned data:")
print(corpus.info())
print("\nFirst 10 rows of the cleaned data:")
print(corpus.head(10))

# Lưu lại bộ corpus đã dọn
corpus.to_csv('data/cleaned_mhc.csv', index=False)

Shape of the cleaned data: (27940, 2)

Info of the cleaned data:
<class 'pandas.core.frame.DataFrame'>
Index: 27940 entries, 0 to 27976
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    27940 non-null  object
 1   label   27940 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 654.8+ KB
None

First 10 rows of the cleaned data:
                                                text  label
0  dear american teen question dutch person heard...      0
1  nothing look forward life dont many reason kee...      1
2  music recommendation im looking expand playlis...      0
3  im done trying feel reason im still alive know...      1
4  worried year old girl subject domestic going l...      1
5  hey rredflag sure right place post go im curre...      1
6  feel like someone need hear tonight feeling ri...      0
7  deserve died right noone would care real frien...      1
8  feel good ive set killing friday nice finally ...   